In [4]:
#Importing Packages I'll need
import pandas as pd
import numpy as np
import os

# Initial Loading

In [2]:
#Loading the main Opportunity Atlas file. This will only work if the file below is in the same folder as this notebook
OA = pd.read_csv("tract_outcomes_early.csv")
OA.head()

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5703) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,state,county,tract,kir_natam_female_p1,kir_natam_female_p25,kir_natam_female_p50,kir_natam_female_p75,kir_natam_female_p100,kir_natam_female_n,kir_natam_female_mean,...,jail_white_pooled_mean_se,kfr_white_pooled_mean_se,staytract_white_pooled_mean_se,staycz_white_pooled_mean_se,stayhome_white_pooled_mean_se,working_white_pooled_mean_se,kir_stycz_white_pooled_mean_se,kfr_stycz_white_pooled_mean_se,spouse_rk_white_pooled_mean_se,par_rank_white_pooled_mean_se
0,1,1,20100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007490,0.014829,0.020353,0.025752,0.019710,0.025348,0.018290,0.017890,0.018882,0.013778
1,1,1,20200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003828,0.020797,0.028821,0.041338,0.028252,0.029613,0.030380,0.028010,0.025328,0.021874
2,1,1,20300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002141,0.011726,0.016670,0.022392,0.015206,0.019161,0.015177,0.015285,0.016145,0.009353
3,1,1,20400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001777,0.010787,0.016027,0.019096,0.012477,0.016844,0.014253,0.014508,0.014396,0.009628
4,1,1,20500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000983,0.008789,0.012861,0.017491,0.010930,0.014396,0.011836,0.012748,0.010329,0.006667


#### Loading the newest Business Master File Data.

In [3]:
#First, 1989
BMF_1989 = pd.read_csv("bmf.bmf89.csv")
BMF_1989.head()

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,EIN,NAME,CITY,STATE,ZIP,SUBSECCD,ZIP5,MSA_NECH,PMSA,fipsold,...,Income,stateOld1,zip5Old1,cityOld1,NteeOldB,Ntee2007a,Confidence2007a,RandNum,nteeFinal,nteeFinal1
0,10024155,BANGOR BAND,BANGOR,ME,04401-5828,3.0,4401,733.0,NaN,NaN,...,0,ME,4401,BANGOR,Z99,A6C,A,0.36818,A6C,A
1,10024645,BANGOR SYMPHONY ORCHESTRA,BANGOR,ME,04401-1441,3.0,4401,733.0,NaN,NaN,...,0,ME,4401,BANGOR,Z99,A69,A,0.54812,A69,A
2,10078060,HENRIETTA D GOODALL HOSPITAL INC,SANFORD,ME,04073-2621,3.0,4073,NaN,NaN,NaN,...,0,ME,4073,SANFORD,Z99,E22,A,0.63606,E22,E
3,10130427,NORTHERN CUMBERLAND MEMORIAL HOSPITAL,BRIDGETON,ME,04009-0000,3.0,4009,6403.0,NaN,NaN,...,0,ME,4009,BRIDGETON,Z99,E22,A,0.53593,E22,E
4,10131950,OLD FOLKS HOME IN BATH,BATH,ME,04530-1697,3.0,4530,NaN,NaN,NaN,...,0,ME,4530,BATH,Z99,A80,C,0.77148,A80,A


In [8]:
#Next 2015. I've downloaded all the 2015 files and am combining them to make sure all nonprofits that began and ended
#in 2015 are included.

#Once you've run this cell, don't run it again

#This makes an empty table
BMF_2015 = pd.DataFrame([])

#This goes through the individual files and adds them to the table
for filename in os.listdir('2015_bmf'):
     BMF_2015 = BMF_2015.append(pd.read_csv('2015_bmf/'+filename, low_memory=False))

BMF_2015.head()

,ACCPER,ADDRESS,ASSETS,CITY,EIN,EPOST,FILER,FIPS,FISYR_IMAGE,FNDNCD,...,cFinSrc,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5
0,6.0,1706 GREENLEAF DR,0.0,NORFOLK,3154.0,Z,N,51710.0,NaN,15.0,...,NaN,NaN,NaN,N,A,C36,C,EN,EN,OT
1,12.0,1050 THORNDIKE ST,0.0,PALMER,19818.0,Z,N,25013.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT
2,12.0,523 E BROADWAY,0.0,SOUTH BOSTON,29215.0,Z,N,25025.0,NaN,10.0,...,NaN,NaN,NaN,N,C,X20,X,RE,RE,OT
3,12.0,PO BOX 92,0.0,HOSFORD,260049.0,Z,N,12077.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT
4,12.0,PO BOX 296,0.0,LABELLE,490336.0,Z,N,12051.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT


In [23]:
#Since the same org will appear several times, this will get rid of duplicates
BMF_2015.drop_duplicates(inplace = True)

#### The Business Master File is more granular than Opportunity Atlas, so we have to match BMF to the minimum shared granularity: census tract.

This will require some in-between tables: zipcode to census tract. Unfortunately, the oldest version I found of this table is first quarter 2010 which may differ from 1989. However, until we find another, it will have to do.

In [26]:
z_to_c_2010 = pd.read_excel("ZIP_TRACT_032010.xlsx").iloc[:, [0,1]]
z_to_c_2015 = pd.read_excel("ZIP_TRACT_122015.xlsx").iloc[:, [0,1]]
z_to_c_2015.head()

,ZIP,TRACT
0,501,36103158607
1,601,72001956600
2,601,72001956700
3,601,72001956300
4,602,72003430200


Here I'm merging the tables together and getting rid of the address information

In [1]:
BMF_1989 = BMF_1989.merge(z_to_c_2010, how = "left", left_on = "ZIP5", right_on = "ZIP")
BMF_1989.drop(["CITY", "STATE", "ZIP_x", "ZIP5", "ZIP_y"], axis = 1, inplace = True)

NameError: name 'BMF_1989' is not defined